In [16]:
import pandas as pd
from collections import Counter
import ast
import re
import json
from sklearn.ensemble import RandomForestClassifier


C:\Users\B-17\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
credits=pd.read_csv('data/the-movies-dataset/credits.csv')
movies=pd.read_csv('data/the-movies-dataset/movies_metadata.csv')

C:\Users\B-17\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1. Merge Credits & Movies

In [4]:
credits.info()
credits['id']=credits['id'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
cast    45476 non-null object
crew    45476 non-null object
id      45476 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [5]:
me=pd.merge(credits, movies, left_on='id', right_on='id', how="right").drop_duplicates()
me.shape

(45466, 26)

In [7]:
dict_columns = ['belongs_to_collection', 'genres', 'production_companies',
                'production_countries', 'spoken_languages', 'cast', 'crew']
def text_to_dict(df):
    for column in dict_columns:
        df[column] = df[column].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x) )
    return df

me = text_to_dict(me)

# 2. Make new Column ( genres, spoken_language, cast)

In [8]:
me['num_genres'] = me['genres'].apply(lambda x: len(x) if x != {} else 0)
me['num_genres']
me['all_genres'] = me['genres'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')


list_of_genres = list(me['genres'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
top_genres = [m[0] for m in Counter([i for j in list_of_genres for i in j]).most_common(15)]
for g in top_genres:
    me['genre_' + g] = me['all_genres'].apply(lambda x: 1 if g in x else 0)



me['all_casts'] = me['cast'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
list_of_casts = list(me['cast'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
top_casts = [m[0] for m in Counter([i for j in list_of_casts for i in j]).most_common(30)]
for c in top_casts:
    me['cast_' + c] = me['all_casts'].apply(lambda x: 1 if c in x else 0)

#all_spoken_languages
me['all_spoken_languages'] = me['spoken_languages'].apply(lambda x: ' '.join(sorted([i['iso_639_1'] for i in x])) if x != {} else '')
list_of_casts = list(me['spoken_languages'].apply(lambda x: [i['iso_639_1'] for i in x] if x != {} else []).values)
top_casts = [m[0] for m in Counter([i for j in list_of_casts for i in j]).most_common(15)]
for c in top_casts:
    me['spoken_language_' + c] = me['all_casts'].apply(lambda x: 1 if c in x else 0)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45466 entries, 0 to 45541
Data columns (total 90 columns):
cast                       45466 non-null object
crew                       45466 non-null object
id                         45466 non-null object
adult                      45466 non-null object
belongs_to_collection      45466 non-null object
budget                     45466 non-null object
genres                     45466 non-null object
homepage                   7780 non-null object
imdb_id                    45449 non-null object
original_language          45455 non-null object
original_title             45466 non-null object
overview                   44512 non-null object
popularity                 45461 non-null object
poster_path                45080 non-null object
production_companies       45466 non-null object
production_countries       45466 non-null object
release_date               45379 non-null object
revenue                    45460 non-null float64
runtime  

# 3. NaN값 제거

In [9]:
me=me[(me['cast']!="[]") & (me['crew']!="[]")]

drop_columns=['homepage','poster_path','tagline','video']
me.drop(drop_columns, axis=1, inplace=True)

me=me[me['imdb_id'].notnull()]
me['overview'].fillna("", inplace=True)
me=me[me['popularity'].notnull()]
me['release_date']=me['release_date'].fillna("")

drop_rows=['original_language','revenue','spoken_languages','title','vote_average','vote_count']
for d in drop_rows:
    me=me[me[d].notnull()]
me=me[me['status']=='Released']
me=me[me['runtime'].notnull()]
me['belongs_to_collection'].fillna("{}", inplace=True)

# 4. Date 나누기

In [10]:
# release_date - Release date of a movie in mm/dd/yy format
date=me['release_date'].str.split('-',expand=True)
me['release_month']=date.iloc[:,1]
me['release_day']=date.iloc[:,2]
me['release_year']=date.iloc[:,0]


# 5. make new columns to Commany Values

In [11]:
#production_companies
me['all_production_companies'] = me['production_companies'].apply(lambda x: ' '.join(sorted([i['name'] for i in x])) if x != {} else '')
list_of_casts = list(me['production_companies'].apply(lambda x: [i['name'] for i in x] if x != {} else []).values)
top_casts = [m[0] for m in Counter([i for j in list_of_casts for i in j]).most_common(20)]
for c in top_casts:
    me['production_company_' + c] = me['all_casts'].apply(lambda x: 1 if c in x else 0)



# 6. 컬럼확인

In [25]:
i = 1
for column in list(me.columns):
    print("\""+column+"\", ", end='')
    i = i + 1

"cast", "crew", "id", "adult", "belongs_to_collection", "budget", "genres", "imdb_id", "original_language", "original_title", "overview", "popularity", "production_companies", "production_countries", "release_date", "revenue", "runtime", "spoken_languages", "status", "title", "vote_average", "vote_count", "num_genres", "all_genres", "genre_Drama", "genre_Comedy", "genre_Thriller", "genre_Romance", "genre_Action", "genre_Horror", "genre_Crime", "genre_Documentary", "genre_Adventure", "genre_Science Fiction", "genre_Family", "genre_Mystery", "genre_Fantasy", "genre_Animation", "genre_Foreign", "all_casts", "cast_Bess Flowers", "cast_Christopher Lee", "cast_John Wayne", "cast_Samuel L. Jackson", "cast_Gérard Depardieu", "cast_Michael Caine", "cast_John Carradine", "cast_Jackie Chan", "cast_Donald Sutherland", "cast_Frank Welker", "cast_Robert De Niro", "cast_Danny Trejo", "cast_Steve Buscemi", "cast_Irving Bacon", "cast_John Goodman", "cast_Christopher Walken", "cast_Danny Glover", "cast_

# 7. Budget 채우기

In [15]:
me['budget']=me['budget'].astype(int)    
me.corr()

,budget,revenue,runtime,vote_average,vote_count,num_genres,genre_Drama,genre_Comedy,genre_Thriller,genre_Romance,...,production_company_New Line Cinema,production_company_Walt Disney Pictures,production_company_Touchstone Pictures,production_company_TriStar Pictures,production_company_Miramax Films,production_company_Mosfilm,production_company_France 2 Cinéma,production_company_Centre National de la Cinématographie (CNC),production_company_Toho Company,production_company_BBC Films
budget,1.000000,0.768645,0.134861,0.073055,0.676398,0.193625,-0.040364,0.012370,0.078700,-0.008705,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
revenue,0.768645,1.000000,0.103910,0.084082,0.811939,0.141722,-0.046569,0.014417,0.040259,-0.002209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
runtime,0.134861,0.103910,1.000000,0.157733,0.113443,0.153976,0.209455,-0.046636,0.051267,0.090392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vote_average,0.073055,0.084082,0.157733,1.000000,0.124182,0.111514,0.132632,0.031248,-0.001787,0.039241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vote_count,0.676398,0.811939,0.113443,0.124182,1.000000,0.146112,-0.025320,-0.001216,0.071092,-0.012333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
num_genres,0.193625,0.141722,0.153976,0.111514,0.146112,1.000000,0.253212,0.143780,0.370300,0.273672,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
genre_Drama,-0.040364,-0.046569,0.209455,0.132632,-0.025320,0.253212,1.000000,-0.164807,0.002279,0.184846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
genre_Comedy,0.012370,0.014417,-0.046636,0.031248,-0.001216,0.143780,-0.164807,1.000000,-0.204341,0.159386,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
genre_Thriller,0.078700,0.040259,0.051267,-0.001787,0.071092,0.370300,0.002279,-0.204341,1.000000,-0.106298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
genre_Romance,-0.008705,-0.002209,0.090392,0.039241,-0.012333,0.273672,0.184846,0.159386,-0.106298,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def predict_bg(data):
    dataBudget0=data[data['budget']==0]
    dataBudgetNot0=data[data['budget']!=0]
    
    # 변수들 -> 예산 예측
    xdata=['revenue','vote_count','runtime','genre_Action','genre_Adventure','genre_Fantasy']
    dataBudgetNot0['budget']=dataBudgetNot0['budget'].astype('str')
    
    # 랜덤포레스트 분류기
    rfm_budget=RandomForestClassifier()
    rfm_budget.fit(dataBudgetNot0[xdata],dataBudgetNot0['budget'])
    budget0Values=rfm_budget.predict(dataBudget0[xdata])
    
    preBudget0=dataBudget0
    preBudgetNot0=dataBudgetNot0
    preBudget0['budget']=budget0Values
    data=preBudgetNot0.append(preBudget0)
    return data

me=predict_bg(me)
me

C:\Users\B-17\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\B-17\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,cast,crew,id,adult,belongs_to_collection,budget,genres,imdb_id,original_language,original_title,...,production_company_New Line Cinema,production_company_Walt Disney Pictures,production_company_Touchstone Pictures,production_company_TriStar Pictures,production_company_Miramax Films,production_company_Mosfilm,production_company_France 2 Cinéma,production_company_Centre National de la Cinématographie (CNC),production_company_Toho Company,production_company_BBC Films
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",tt0114709,en,Toy Story,...,0,0,0,0,0,0,0,0,0,0
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,False,{},65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",tt0113497,en,Jumanji,...,0,0,0,0,0,0,0,0,0,0
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,False,{},16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0114885,en,Waiting to Exhale,...,0,0,0,0,0,0,0,0,0,0
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949,False,{},60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",tt0113277,en,Heat,...,0,0,0,0,0,0,0,0,0,0
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860,False,{},58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",tt0114319,en,Sabrina,...,0,0,0,0,0,0,0,0,0,0
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091,False,{},35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",tt0114576,en,Sudden Death,...,0,0,0,0,0,0,0,0,0,0
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",tt0113189,en,GoldenEye,...,0,0,0,0,0,0,0,0,0,0
10,"[{'cast_id': 1, 'character': 'Andrew Shepherd'...","[{'credit_id': '52fe44dac3a36847f80adfa3', 'de...",9087,False,{},62000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0112346,en,The American President,...,0,0,0,0,0,0,0,0,0,0
13,"[{'cast_id': 1, 'character': 'Richard Nixon', ...","[{'credit_id': '52fe43c59251416c7501d6f3', 'de...",10858,False,{},44000000,"[{'id': 36, 'name': 'History'}, {'id': 18, 'na...",tt0113987,en,Nixon,...,0,0,0,0,0,0,0,0,0,0
14,"[{'cast_id': 1, 'character': 'Morgan Adams', '...","[{'credit_id': '52fe42f4c3a36847f802f69f', 'de...",1408,False,{},98000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",tt0112760,en,Cutthroat Island,...,0,0,0,0,0,0,0,0,0,0


In [20]:
me['budget'].value_counts()

2000000      2343
500000       2035
1000000      1771
3000000      1474
5000000      1400
10000000     1192
1500000       960
4000000       740
6000000       654
250000        623
12000000      619
20000000      613
15000000      602
3500000       556
2500000       552
1200000       536
7000000       530
8000000       510
200000        503
1             501
100000        499
150000        454
700000        437
300000        430
600000        379
25000000      373
30000000      350
25000         346
75000         328
800000        327
             ... 
32350000        1
18550417        1
82500000        1
5402000         1
26500000        1
5985            1
2159280         1
15774948        1
8520000         1
163000000       1
9272437         1
485000          1
1288000         1
1948000         1
445000          1
46630000        1
686             1
2250000         1
11930000        1
2742651         1
98              1
2053648         1
78146652        1
13360000        1
5210000   

In [22]:
me.columns

Index(['cast', 'crew', 'id', 'adult', 'belongs_to_collection', 'budget',
       'genres', 'imdb_id', 'original_language', 'original_title',
       ...
       'production_company_New Line Cinema',
       'production_company_Walt Disney Pictures',
       'production_company_Touchstone Pictures',
       'production_company_TriStar Pictures',
       'production_company_Miramax Films', 'production_company_Mosfilm',
       'production_company_France 2 Cinéma',
       'production_company_Centre National de la Cinématographie (CNC)',
       'production_company_Toho Company', 'production_company_BBC Films'],
      dtype='object', length=110)

In [40]:
castList=["cast_Bess Flowers", "cast_Christopher Lee", "cast_John Wayne", "cast_Samuel L. Jackson", "cast_Gérard Depardieu", "cast_Michael Caine", "cast_John Carradine", "cast_Jackie Chan", "cast_Donald Sutherland", "cast_Frank Welker", "cast_Robert De Niro", "cast_Danny Trejo", "cast_Steve Buscemi", "cast_Irving Bacon", "cast_John Goodman", "cast_Christopher Walken", "cast_Danny Glover", "cast_Dennis Hopper", "cast_Susan Sarandon", "cast_John Hurt", "cast_Eric Roberts", "cast_Bruce Willis", "cast_Malcolm McDowell", "cast_Harvey Keitel", "cast_Willem Dafoe", "cast_Anthony Quinn", "cast_Keith David", "cast_Donald Pleasence", "cast_Stellan Skarsgård", "cast_Morgan Freeman"]
#me[ me[lambda x: c for cast in castList]==0 ]
me.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44741 entries, 0 to 45541
Columns: 110 entries, cast to production_company_BBC Films
dtypes: float64(4), int64(81), object(25)
memory usage: 39.1+ MB
